# Analyse pyBadlands stratigraphic output

If the stratigraphic structure is turned on in the XmL input file, **pyBadlands** produces sedimentary layers Hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the stratigraphic architecture in an IPython notebook.

In [ ]:
import os
import re
import rdp
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt

import stratalAnalyse as strata

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# 1-  Visualize predicted stratigraphic layers at the final step

## Loading the stratigraphic file

First we load a single stratigraphic file. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.p`X`.hdf5`

with T the display time index and X the number of the partition (used in the parallel version). In cases where you ran your simulation in parallel you will also need to give the number of CPUs used (_cpus_).

To load a file you will need to give the folder name and the number of processors used in your simulation.

For more information regarding the function uncomment the following line.

In [ ]:
# help(strata.stratalSection.__init__)

In [ ]:
folder = '/workspace/volume/JSR-models/diff09_sea3c/h5/'
strat = strata.stratalSection(folder)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention). Here, T = 300.

In [ ]:
# help(strat.loadStratigraphy)

In [ ]:
strat.loadStratigraphy(300) 

## Building a cross-section

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide:
+ the position of the segment in the simulation space _(xo,yo)_ and _(xm,ym)_, 
+ a gaussian filter value for smoothing (_gfilt_ a value of 0 can be used for non-smoothing) and 
+ the resolution of the cross-section (based on a number of points: _nbpts_). 

In [ ]:
# Build a cross-section along X axis
x1 = strat.xi.min()
x2 = strat.xi.max()
y1 = 100000
y2 = 100000

# Interpolation parameters
nbpts = strat.nx
gfilt = 1

In [ ]:
# help(strat.buildSection)

In [ ]:
strat.buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

## Visualize the stratal stacking pattern

We use **plotly** to visualise the vertival cross-section of stratal stacking apttern.

In [ ]:
# help(strata.viewSection)

In [ ]:
strata.viewSection(width = 800, height = 500, cs = strat, 
            dnlay = 5, rangeX=[140000, 250000], rangeY=[-800,50],
            linesize = 0.3, title='Stratal stacking pattern coloured by time')

# 2- Analyze predicted stratal architecture based on successive outputs

The stratigraphic analysis in this section include: 

2.1 reconstruct the stratal architecture (stratal stacking pattern, Wheeler diagram, and vertical stacking pattern);

2.2 interpret the synthetic depositional cycles using trajectory analysis ([Helland-Hansen & Hampson (2009)](http://onlinelibrary.wiley.com/wol1/doi/10.1111/j.1365-2117.2009.00425.x/full)) and the accommodation succession method ([Neal & Abreu (2009)](https://www.researchgate.net/publication/249521744_Sequence_stratigraphy_hierarchy_and_the_accommodation_succession_method)).

By reading successive outputs, we calculate the **shorleine trajectory, shelf-edge trajectory, changes in accommodation creation $\delta A$ and sedimentation $\delta S$**.

In this example, 300 outputs are recorded (**`nbout`**). We use a loop to read the output every 5 timesteps (**`nstep`**). 

In [ ]:
# read multi outputs
folder = '/workspace/volume/JSR-models/diff09_sea3c/h5/'
strat_all = {}
nbout = 300 
nstep = 5
# read the first layer
strat_all[0] = strata.stratalSection(folder)
strat_all[0].loadStratigraphy_multi(1,nstep)
k = 1
for i in range(nstep,nbout+1,nstep):
    strat_all[k] = strata.stratalSection(folder)
    strat_all[k].loadStratigraphy_multi(i,nstep)
    k += 1

## Build cross-section for each sedimentary layer

In [ ]:
nbcs = len(strat_all)
for i in range(0,nbcs):
    strat_all[i].buildSection_multi(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

## Read the sea-level value for each stratigraphic layer

In [ ]:
# Read sea-level file
sea = []
time = []
step=300
folder1 = '/workspace/volume/JSR-models/diff09_sea3c/'
for  k in range(0, step+1):
    with open(folder1+'/xmf/tin.time'+str(k)+'.xmf') as fp: 
        line = fp.readline()
        cnt = 1
        while line:
            if 'Time' in line:
                val = [float(s) for s in re.findall(r'-?\d+\.?\d*', line)]
                time.append(val[0])
            if 'Function' in line:
                val = [float(s) for s in re.findall(r'-?\d+\.?\d*', line)]
                sea.append(val[2])
            line = fp.readline()
            cnt += 1

In [ ]:
# Extract the sea level corresponds to each extracted stratigraphic layer: 1, 10, 20, ..., 300. 31 layers in total.
Sealevel = np.zeros(nbcs)
Time = np.zeros(nbcs)
Sealevel[0] = sea[1]
Time[0] = time[1]
p = 1
for i in range(nstep,nbout+1,nstep):
    Sealevel[p] = sea[i]
    Time[p] = time[i]
    p += 1

# View sea level curve
strata.viewData(x0 = Time, y0 = Sealevel, width = 600, height = 400, linesize = 3, 
                color = '#6666FF',title='Sea-level curve',xlegend='Time [yr]',ylegend='Sea-level position [m]')

## Calculate shoreline and shelf-edge trajectories, $\delta A$ and $\delta S$

* shoreline position at a specific time is defined as the position whose elevation equals to the sea level at that time;
* shelf-edge position is defined by a critical slope ($\pi$*0.0001);
* $\delta A$ is calculated as changes in water depth;
$\delta S$ is calculated as sedimentation rate.

See the figure below:
<img src="images/Traj_AS.png" alt="Trajectory and AS"  width="500" height="400" border="10" />

In [ ]:
# help(strat.buildParameters)

In [ ]:
strat.buildParameters(CS = strat_all, Sealevel = Sealevel, min_angle = np.pi*0.0001)

# 2.1- Reconstruct the stratal architecture

## Build paleo-depositional environment structure

We define paleo-depositional environments (depositional facies) based on deposition depth, and colour the depositional facies differently. For example,

| Facies  | **coastal plain** | **shoreface** | **shelf** | **upper slope** | **middle slope** | **lower slope** | **deep marine** |
| :-----------: |:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|
|Paleo-depth/m| above sea level | 0~10 | 10~50 | 50~200 | 200~500 | 500~800 | >800 |
|Colour| limegreen | darkkhaki | sandybrown | khaki | c | teal | teal |

In [ ]:
# Specify the range of water depth for each depositional environment
depthIDs = [-10, 0, 10, 50, 200, 500, 800]
colors = ['limegreen','darkkhaki','sandybrown','khaki','c','teal','teal','white']

# Get the position of depositional environments on each extracted layer
enviID = np.zeros((len(depthIDs), nbcs))
for i in range(nbcs):
    enviID[:,i] = strata.depthID(cs = strat_all[i], sealevel = Sealevel[i], depthIDs = depthIDs)

# Add the endpoint of deposition to the depositional environments extent
EnviID = np.append(enviID, [strat.depoend], axis=0).astype(int)
EnviID_dist = strat.dist[EnviID]

## Plot stratal stacking pattern

It takes a bit long...

In [ ]:
strata.viewStrata(width = 9, height = 4, cs = strat, enviID = EnviID, dnlay = nstep, colors = colors,
                      rangeX = [140000, 250000], rangeY = [800, -50], linesize = 0.3, title = 'Cross section')

## Plot Wheeler diagram

Wheeler diagram (or chronostratigraphic chart) is a powerful tool to document unconformities between sequences, and to understand the stacking patterns of sedimentary successions and their relationship to sea level. It displays the horizontal distribution of contemporaneous sedimentary layer sequences, as well as hiatuses in sedimentation. 

In [ ]:
timeMA = [x/1e6 for x in Time]  # change the unit to be Myr

strata.viewWheeDiag(width = 9, height = 4, cs = strat, enviID = EnviID_dist, dnlay = nstep, time = timeMA, 
                    colors = colors, rangeX = [140000, 250000], rangeY = [0,30], 
                    dt_height = 0.9, title = 'Wheeler Diagram')

## Plot vertical stacking patterns (synthetic wells)

To plot the vertical stacking at one location, we need the thickness of each stratigraphic layer and its depositional facies.

In [ ]:
# Define the location of wells on the cross-section (m)
posit = np.array([160000, 180000, 200000, 220000, 240000])
# The index of these well locations
positID = (posit/strat.dx).astype(int) + 1
color = ['white','limegreen','darkkhaki','sandybrown','khaki','c','teal']

# Extract the thickness of each layer for each well
layth = []
for m in positID:
    nz = strat.nz-1
    layth.append(strat.secDep[nz][m])
    for n in range(1,nbcs-1):
        layth.append(-sum(strat.secTh[(nz-n*nstep):(nz-(n-1)*nstep)])[m])
layTh = np.reshape(layth, (len(positID), nbcs-1))

# Build the structure of depositional facies corresponded to colors
color_fill = []
for i in positID:
    p = 0
    for j in range(0,strat.nz,nstep):
        if (strat.secElev[j][i]) > (- depthIDs[0]):
            color_fill.append(color[0])
        elif (strat.secElev[j][i]) > (- depthIDs[1]):
            color_fill.append(color[0])
        elif (strat.secElev[j][i]) > (- depthIDs[2]):
            color_fill.append(color[1])
        elif (strat.secElev[j][i]) > (- depthIDs[3]):
            color_fill.append(color[2])
        elif (strat.secElev[j][i]) > (- depthIDs[4]):
            color_fill.append(color[3])
        elif (strat.secElev[j][i]) > (- depthIDs[5]):
            color_fill.append(color[4])
        else:
            color_fill.append(color[5])
        p+=1
colorFill = np.reshape(color_fill, (len(positID), p))

In [ ]:
# Plot vertical stacking (wells)
strata.viewStack(width = 4, height = 5, layTh = layTh, colorFill = colorFill)

# 2.2-  Stratigraphic interpretations

Based on calculated shoreline and shelf-edge trajectories, $\delta A$ and $\delta S$, we define trajectory classes and stacking patterns accordingly, as illustrated in the figure below: <img src="images/Interpretation.png" alt="Interpretation methods"  width="700" height="400" border="10" />

## 2.2.1- Shoreline trajectory analysis and interpretations

## Visualize shoreline trajectory and its gradient

In [ ]:
dispTime = 1e5
xval = Time/dispTime
yval = strat_all[0].dist[strat.shoreID]
gval = np.gradient(yval)

# View shoreline position through time
strata.viewData(x0 = xval, y0 = yval, width = 800, height = 500, linesize = 3, color = '#6666FF',
               title='Shoreline trajectory',xlegend='display step',ylegend='shoreline position in metres')

# Define the gradient evolution
strata.viewData(x0 = xval, y0 = gval, width = 800, height = 500, linesize = 3, color = '#f4a142',
               title='Shoreline trajectory gradient',xlegend='display step',ylegend='gradient shoreline position')

In [ ]:
# Default color used: 
TTC = 'rgba(56,110,164,0.8)'
DRTC = 'rgba(60,165,67,0.8)'  
ARTC = 'rgba(112,54,127,0.8)'
STC = 'rgba(252,149,7,0.8)'

# You can change them by specifying new values in the function
STcolors_ST = strata.build_ShoreTrajectory(xval,yval,gval,Sealevel,nbout,
                                           cTC=TTC,cDRC=DRTC,cARC=ARTC,cSTC=STC)

## Plotting the resulting shoreline trajectory classes

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs = strat, colors = STcolors_ST, 
                     dnlay = 5, rangeX=[140000, 250000], rangeY=[-800,50], 
                     linesize = 0.5, title = 'Classes interpreted based on shoreline trajectory (ST)')

## 2.2.2- Accommodation succession analysis and interpretations

## Visualize $\delta A$ and $\delta S$

In [ ]:
xval = Time/dispTime
ASval = np.array(strat.accom_shore)-np.array(strat.sed_shore)
gval = np.gradient(ASval)

# Accommodation (A) and sedimentation (S) change differences
strata.viewData(x0 = xval, y0 = ASval, width = 800, height = 500, linesize = 3, 
                color = '#6666FF',title='Change between accomodation & sedimentation',xlegend='display step',
                ylegend='A-S')

# Define the gradient evolution
strata.viewData(x0 = xval, y0 = gval, width = 800, height = 500, linesize = 3, color = '#f4a142',
               title='A&S gradient',xlegend='display step',ylegend='gradient A&S')

In [ ]:
# Default color used: 
R = 'rgba(51,79,217,0.8)' 
PA = 'rgba(252,149,7,0.8)' 
APD= 'rgba(15,112,2,0.8)'

# You can change them by specifying new values in the function
STcolors_AS = strata.build_AccomSuccession(xval,ASval,gval,nbout,cR=R,cPA=PA,cAPD=APD)

## Plotting the resulting accommodation succession sequence sets

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs = strat, colors = STcolors_AS, dnlay = 5, 
                     rangeX=[140000, 250000], rangeY=[-800,50], linesize = 0.5, 
                     title = 'Sequence sets interpreted based on change of accommodation and sedimentation (AS)')